In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

In [2]:
from google.colab import drive
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:

%cd /gdrive
%cd My\ Drive
%cd CSCE421YaleData 
!ls

/gdrive
/gdrive/My Drive
/gdrive/My Drive/CSCE421YaleData
csce_421_final_subject_class_emotion.h5    emotions		  subjects
csce_421_final_subject_class.h5		   glasses
csce_421_final_subject_class_wo_imgAug.h5  Homework5_YaleDataset


In [0]:
import os, shutil

original_dataset_dir = '/gdrive/My Drive/CSCE421YaleData/Homework5_YaleDataset'
base_dir = '/gdrive/My Drive/CSCE421YaleData/emotions'



In [0]:
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')
test_dir = os.path.join(base_dir,'test')




In [12]:
from keras import layers
from keras import models


"""
======Model ======
"""
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

"""add dropout to fight overfitting"""
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(4, activation = 'softmax'))
model.summary()
from keras import optimizers
model.compile(loss='categorical_crossentropy', 
              optimizer = optimizers.RMSprop(lr=1e-4),
              metrics = ['acc'])
                        






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         


In [13]:
from keras.preprocessing.image import ImageDataGenerator

"""Model Two: add data augmentation to generate more samples"""

train_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

"""Model Two: change batch_size from 20 to 32"""
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size =(150,150), 
    batch_size=32, 
    class_mode='categorical') 

"""Model Two: change batch_size from 20 to 32"""
validation_generator = test_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150), 
    batch_size=32, 
    class_mode='categorical')

for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

Found 28 images belonging to 4 classes.
Found 16 images belonging to 4 classes.
data batch shape: (28, 150, 150, 3)
labels batch shape: (28, 4)


In [14]:

history = model.fit_generator(
    train_generator, 
    steps_per_epoch=50, 
    epochs=50, 
    validation_data=validation_generator, 
    validation_steps=50)

model.save('csce_421_final_subject_class_emotion.h5')


# model.load_weights('csce_421_final_subject_class_emotion.h5')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





50/50 [==============================] - 22s 445ms/step - loss: 1.3936 - acc: 0.2300 - val_loss: 1.3833 - val_acc: 0.2500
Epoch 2/50
50/50 [==============================] - 15s 297ms/step - loss: 1.3880 - acc: 0.2557 - val_loss: 1.3862 - val_acc: 0.2500
Epoch 3/50
50/50 [==============================] - 15s 297ms/step - loss: 1.3885 - acc: 0.2393 - val_loss: 1.3878 - val_acc: 0.3125
Epoch 4/50
50/50 [==============================] - 15s 299ms/step - loss: 1.3872 - acc: 0.2393 - val_loss: 1.3845 - val_acc: 0.1875
Epoch 5/50
50/50 [==============================] - 15s 297ms/step - loss: 1.3871 - acc: 0.2407 - val_loss: 1.3846 - val_acc: 0.3125
Epoch 6/50
50/50 [==============================] - 15s 295ms/step - loss: 1.3859 - acc: 0.2500 - val_loss: 1.3876 - val_acc: 0.2500
Epoch 7/50
50/50 [==============================] - 15s 295ms/step - loss: 1.3863 - acc: 0.2564 - va

In [15]:
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150,150), 
    batch_size=32, 
    class_mode='categorical')

model.evaluate(test_generator)

Found 16 images belonging to 4 classes.
1/1 [==============================] - 5s 5s/step


[1.1681716442108154, 0.625]